<a href="https://colab.research.google.com/github/marziyemahmoudifar/test2/blob/new1/apple_detection_faster_rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

implementation of apple detection

In [1]:
%tensorflow_version 2.x
%matplotlib inline

import library

In [2]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from skimage import io
from shutil import copyfile
import sys
import time

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [3]:
from google.colab import drive
root = '/content/gdrive/'
drive.mount( root )

Mounted at /content/gdrive/


In [4]:
!unzip gdrive/MyDrive/KFuji_RGB-DS_dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: row data/BD11_inf_201710_138_pc.mat  
  inflating: row data/BD11_inf_201710_138_RGB.jpg  
  inflating: row data/BD11_sup_201710_004_pc.mat  
  inflating: row data/BD11_sup_201710_004_RGB.jpg  
  inflating: row data/BD11_sup_201710_007_pc.mat  
  inflating: row data/BD11_sup_201710_007_RGB.jpg  
  inflating: row data/BD11_sup_201710_010_pc.mat  
  inflating: row data/BD11_sup_201710_010_RGB.jpg  
  inflating: row data/BD11_sup_201710_013_pc.mat  
  inflating: row data/BD11_sup_201710_013_RGB.jpg  
  inflating: row data/BD11_sup_201710_037_pc.mat  
  inflating: row data/BD11_sup_201710_037_RGB.jpg  
  inflating: row data/BD11_sup_201710_043_pc.mat  
  inflating: row data/BD11_sup_201710_043_RGB.jpg  
  inflating: row data/BD11_sup_201710_046_pc.mat  
  inflating: row data/BD11_sup_201710_046_RGB.jpg  
  inflating: row data/BD11_sup_201710_049_pc.mat  
  inflating: row data/BD11_sup_201710_049_RGB.jpg  
  inflating: row data/

In [6]:
DATADIR = "preprocessed data"
CATEGORIES = os.listdir(DATADIR)
print(CATEGORIES)
CATEGORIES = ['BACKGROUND_Google', 'Motorbikes', 'airplanes', 'Faces_easy', 'Faces']
print(CATEGORIES)

['images', 'square_annotations1', 'annotations', 'sets']
['BACKGROUND_Google', 'Motorbikes', 'airplanes', 'Faces_easy', 'Faces']
